In [1]:
from training_functions import train_model
from data_helper_functions import get_tournament_ids_by_year


tournament_ids = get_tournament_ids_by_year(2023)

/Users/sam/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
model = train_model(tournament_ids[:30], False, True)

Saving weights to JSON.

Accuracy Score: 0.9065804935370153



In [8]:
import pandas as pd

df = pd.read_csv("data/df_csv/out.csv", index_col=[0])

In [9]:
df

,level,shutdownValue,totalGold,MINIONS_KILLED,NEUTRAL_MINIONS_KILLED,NEUTRAL_MINIONS_KILLED_YOUR_JUNGLE,NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE,CHAMPIONS_KILLED,NUM_DEATHS,ASSISTS,...,TOTAL_DAMAGE_DEALT_TO_CHAMPIONS,TOTAL_DAMAGE_TAKEN,TOTAL_DAMAGE_SELF_MITIGATED,TOTAL_DAMAGE_SHIELDED_ON_TEAMMATES,TOTAL_DAMAGE_DEALT_TO_BUILDINGS,TOTAL_DAMAGE_DEALT_TO_TURRETS,TOTAL_DAMAGE_DEALT_TO_OBJECTIVES,TOTAL_TIME_CROWD_CONTROL_DEALT_TO_CHAMPIONS,TOTAL_HEAL_ON_TEAMMATES,RESULT
0,14.0,0.0,8978.0,190.0,0.000000,0.000000,0.000000,1.0,6.0,0.0,...,10447.824219,21855.882812,25677.835938,0.000000,3834.446045,3834.446045,5834.885742,57.047157,0.000000,0.0
1,11.0,0.0,7002.0,4.0,105.000015,77.000015,12.000001,1.0,1.0,3.0,...,5543.689453,6642.986816,4700.005859,6867.362305,0.000000,0.000000,1741.593140,43.024811,240.000000,0.0
2,14.0,0.0,9444.0,227.0,0.000000,0.000000,0.000000,3.0,4.0,1.0,...,17999.410156,19635.832031,12220.982422,0.000000,0.000000,0.000000,0.000000,24.111908,0.000000,0.0
3,12.0,0.0,10108.0,197.0,8.000000,8.000000,0.000000,2.0,4.0,0.0,...,15108.281250,12588.553711,9689.991211,0.000000,815.076111,815.076111,1293.170654,27.441101,0.000000,0.0
4,10.0,0.0,5820.0,41.0,0.000000,0.000000,0.000000,0.0,7.0,4.0,...,2762.619629,12969.580078,19943.939453,0.000000,0.000000,0.000000,325.012482,67.131958,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,17.0,0.0,13177.0,208.0,4.500000,4.000000,0.000000,4.0,6.0,8.0,...,10908.503906,31829.662109,54254.277344,356.949005,492.374237,492.374237,3326.256348,270.043121,0.000000,1.0
986,16.0,0.0,12052.0,46.0,130.000031,75.000031,10.500000,2.0,5.0,12.0,...,16442.261719,49656.523438,79353.898438,0.000000,0.000000,0.000000,30863.296875,97.073639,4641.274902,1.0
987,18.0,0.0,17116.0,324.0,31.999998,24.999998,2.500000,3.0,5.0,5.0,...,28132.595703,24659.025391,21580.189453,0.000000,10637.882812,10637.882812,29549.302734,63.020660,0.000000,1.0
988,18.0,0.0,19800.0,335.0,35.999996,30.499996,0.000000,11.0,4.0,6.0,...,49881.511719,23351.023438,19703.501953,0.000000,1458.057861,1458.057861,8589.797852,17.434143,1086.000000,1.0
